# Tests for DataMux

In [1]:
import logging
logging.basicConfig(format="%(asctime)-15s %(message)s", level=logging.INFO)

In [2]:
from dotenv import load_dotenv
load_dotenv()

True

In [3]:
from serving import WebsocketClient

server_host = "0.0.0.0"
server_port = 3300

client = WebsocketClient(server_host, server_port)
await client.connect()

2023-07-06 22:52:04,281 Connected to DataMux: ws://0.0.0.0:3300/ws


In [4]:
import asyncio

collections = None
collection_streams = None
live_streams = None

async def log_messages():

    global collections
    global collection_streams
    global live_streams

    while True:
        topic, content = await client.recvmsg()
        if topic == client.protocol.LIST_COLLECTIONS:
            collections = content["collections"]
        if topic == client.protocol.LIST_COLLECTION_STREAMS:
            collection_streams = content["streams"]
        if topic == client.protocol.LIST_LIVE_STREAMS:
            live_streams = content["streams"]
logging_task = asyncio.create_task(log_messages())

In [5]:
await client.sendmsg(*client.datamux.list_collections())

2023-07-06 22:52:08,601 topic: b'list_collections'
2023-07-06 22:52:08,603 content: {'collections': [{'name': 'adhd_sin', 'description': 'Eye movements of participants during an audiovisual-SIN task', 'keywords': ['SIN', 'ADHD'], 'authors': [{'name': 'Anne M. P. Michalek', 'affiliation': 'Old Dominion University', 'email': 'aperrott@odu.edu'}], 'streams': {'gaze': {'name': 'Gaze', 'description': 'gaze point', 'unit': '', 'frequency': 60.0, 'fields': {'x': {'name': 'gaze x', 'description': 'Normalized X-Gaze Point', 'dtype': 'f32'}, 'y': {'name': 'gaze y', 'description': 'Normalized Y-Gaze Point', 'dtype': 'f32'}}, 'index': {'t': {'name': 'timestamp', 'description': 'Recording Timestamp', 'dtype': 'f32'}}, 'node': {'device': {'model': 'Pro X2-60', 'manufacturer': 'Tobii', 'category': 'Eye Tracker'}, 'uri': None, 'inputs': {}, 'outputs': {}}, 'attrs': {}}, 'pupil': {'name': 'Pupil', 'description': 'pupil diameter', 'unit': 'mm', 'frequency': 60.0, 'fields': {'d': {'name': 'pupil d', 'des

In [6]:
await asyncio.sleep(2)
assert collections
await client.sendmsg(*client.datamux.list_collection_streams(collections[0]["name"]))

2023-07-06 22:52:12,890 topic: b'list_collection_streams'
2023-07-06 22:52:12,903 content: {'collection_name': 'adhd_sin', 'streams': [{'name': 'Gaze', 'description': 'gaze point', 'unit': '', 'frequency': 60.0, 'fields': {'x': {'name': 'gaze x', 'description': 'Normalized X-Gaze Point', 'dtype': 'f32'}, 'y': {'name': 'gaze y', 'description': 'Normalized Y-Gaze Point', 'dtype': 'f32'}}, 'index': {'t': {'name': 'timestamp', 'description': 'Recording Timestamp', 'dtype': 'f32'}}, 'node': {'device': {'model': 'Pro X2-60', 'manufacturer': 'Tobii', 'category': 'Eye Tracker'}, 'uri': None, 'inputs': {}, 'outputs': {}}, 'attrs': {'diagnosis': 'Non-ADHD', 'gender': 'female', 'noise': '0', 'participant': '12', 'question': '10', 'collection': 'adhd_sin', 'id': 'gaze'}}, {'name': 'Pupil', 'description': 'pupil diameter', 'unit': 'mm', 'frequency': 60.0, 'fields': {'d': {'name': 'pupil d', 'description': 'Pupil Diameter', 'dtype': 'f32'}}, 'index': {'t': {'name': 'timestamp', 'description': 'Recor

In [8]:
await asyncio.sleep(2)
assert collections
assert collection_streams
await client.sendmsg(*client.datamux.replay_collection_stream(
    collections[0]["name"],
    collection_streams[0]["name"],
    collection_streams[0]["attrs"]
))

2023-07-06 22:52:40,394 topic: b'replay_collection_stream'
2023-07-06 22:52:40,395 content: {'collection_name': 'adhd_sin', 'stream_name': 'Gaze', 'attrs': {'diagnosis': 'Non-ADHD', 'gender': 'female', 'noise': '0', 'participant': '12', 'question': '10', 'collection': 'adhd_sin', 'id': 'gaze'}, 'status': 1}
2023-07-06 22:52:40,397 topic: b'data'
2023-07-06 22:52:40,398 content: {'index': [0.0], 'value': [973.0, 748.0]}
2023-07-06 22:52:40,411 topic: b'data'
2023-07-06 22:52:40,412 content: {'index': [16.670000076293945], 'value': [976.0, 752.0]}
2023-07-06 22:52:40,428 topic: b'data'
2023-07-06 22:52:40,429 content: {'index': [33.33000183105469], 'value': [975.0, 750.0]}
2023-07-06 22:52:40,446 topic: b'data'
2023-07-06 22:52:40,447 content: {'index': [50.0], 'value': [973.0, 748.0]}
2023-07-06 22:52:40,464 topic: b'data'
2023-07-06 22:52:40,465 content: {'index': [66.66000366210938], 'value': [972.0, 745.0]}
2023-07-06 22:52:40,481 topic: b'data'
2023-07-06 22:52:40,483 content: {'ind

In [9]:
await asyncio.sleep(2)
assert collections
assert collection_streams
await client.sendmsg(*client.datamux.restream_collection_stream(
    collections[0]["name"],
    collection_streams[0]["name"],
    collection_streams[0]["attrs"]
))

2023-07-06 22:53:04,272 topic: b'restream_collection_stream'
2023-07-06 22:53:04,273 content: {'collection_name': 'adhd_sin', 'stream_name': 'Gaze', 'attrs': {'diagnosis': 'Non-ADHD', 'gender': 'female', 'noise': '0', 'participant': '12', 'question': '10', 'collection': 'adhd_sin', 'id': 'gaze'}, 'status': 1}


In [10]:
await client.sendmsg(*client.datamux.list_live_streams())

2023-07-06 22:53:08,176 topic: b'list_live_streams'
2023-07-06 22:53:08,178 content: {'streams': [{'name': 'Gaze', 'description': 'gaze point', 'unit': '', 'frequency': 60.0, 'fields': {'x': {'name': 'gaze x', 'description': 'Normalized X-Gaze Point', 'dtype': 'f32'}, 'y': {'name': 'gaze y', 'description': 'Normalized Y-Gaze Point', 'dtype': 'f32'}}, 'index': {'t': {'name': 'timestamp', 'description': 'Recording Timestamp', 'dtype': 'f32'}}, 'node': {'device': {'model': 'Pro X2-60', 'manufacturer': 'Tobii', 'category': 'Eye Tracker'}, 'uri': None, 'inputs': {}, 'outputs': {}}, 'attrs': {'diagnosis': 'Non-ADHD', 'gender': 'female', 'noise': '0', 'participant': '12', 'question': '10', 'collection': 'adhd_sin', 'id': 'gaze', 'dfds_mode': 'restream'}}]}


In [11]:
await asyncio.sleep(2)
assert live_streams
await client.sendmsg(*client.datamux.relay_live_streams(
    live_streams[0]["name"],
    live_streams[0]["attrs"]
))

2023-07-06 22:53:13,903 topic: b'relay_live_stream'
2023-07-06 22:53:13,905 content: {'stream_name': 'Gaze', 'status': 1}
2023-07-06 22:53:13,926 topic: b'data'
2023-07-06 22:53:13,927 content: {'index': [5128345.519011016], 'value': [[973.0, 748.0]]}
2023-07-06 22:53:13,943 topic: b'data'
2023-07-06 22:53:13,944 content: {'index': [16.670000076293945], 'value': [[976.0, 752.0]]}
2023-07-06 22:53:13,960 topic: b'data'
2023-07-06 22:53:13,962 content: {'index': [33.33000183105469], 'value': [[975.0, 750.0]]}
2023-07-06 22:53:13,978 topic: b'data'
2023-07-06 22:53:13,979 content: {'index': [50.0], 'value': [[973.0, 748.0]]}
2023-07-06 22:53:13,995 topic: b'data'
2023-07-06 22:53:13,996 content: {'index': [66.66000366210938], 'value': [[972.0, 745.0]]}
2023-07-06 22:53:14,012 topic: b'data'
2023-07-06 22:53:14,014 content: {'index': [83.33000183105469], 'value': [[966.0, 747.0]]}
2023-07-06 22:53:14,030 topic: b'data'
2023-07-06 22:53:14,031 content: {'index': [99.95999908447266], 'value'